# Dirac Notation

In [2]:
import numpy as np

def PrettyPrintBinary(myState):
    output = f'('
    for state in myState:
        output= output + str(state[0]) + ' |' + state[1] + '> + '
    output=output[:-3]+')'
    print(output)

def PrettyPrintInteger(myState):
    output = f'('
    for state in myState:
        output= output + str(state[0]) + ' |' + str(int(state[1],2)) + '> + '
    output=output[:-3]+')'
    print(output)

In [3]:
myState2=[
  (np.sqrt(0.1)*1.j, '101'),
  (np.sqrt(0.5), '000') ,
  (-np.sqrt(0.4), '010' )
]
PrettyPrintBinary(myState2)
PrettyPrintInteger(myState2)

(0.31622776601683794j |101> + 0.7071067811865476 |000> + -0.6324555320336759 |010>)
(0.31622776601683794j |5> + 0.7071067811865476 |0> + -0.6324555320336759 |2>)


In [4]:
def DiracToVec(myState):
    vec = [0] * 2**(len(myState[0][1]))
    for state in myState:
        idx = int(state[1],2)
        vec[idx]=state[0]
    return vec

def VecToDirac(myVec):
    output = []
    bin_size = int(np.ceil(np.log2(len(myVec))))
    for i,state in enumerate(myVec):
        if state != 0:
            output.append((state,format(i,f'0{bin_size}b')))    
    return output
print(DiracToVec(myState2))
print(VecToDirac(DiracToVec(myState2)))

[0.7071067811865476, 0, -0.6324555320336759, 0, 0, 0.31622776601683794j, 0, 0]
[(0.7071067811865476, '000'), (-0.6324555320336759, '010'), (0.31622776601683794j, '101')]


# Quantum Simulators
## Simulator S

In [5]:
import itertools

def AddDuplicate(myState):
    seen=dict()
    for state in myState:
        if state[1] in seen:
            seen[state[1]]+=state[0]
        else: seen[state[1]]=state[0]
    return [(amplitude,state) for (state,amplitude) in list(seen.items()) if amplitude!=0]

myState=[
  (-np.sqrt(0.125), '11'),
  (np.sqrt(0.1), '00'),
  (np.sqrt(0.4), '01'),
  (-np.sqrt(0.125), '11')
]

PrettyPrintBinary(AddDuplicate(myState))

(-0.7071067811865476 |11> + 0.31622776601683794 |00> + 0.6324555320336759 |01>)


In [6]:
def bitFlip(idx, bitstring):
    notState = list(bitstring)
    notState[idx] = '1' if bitstring[idx]=='0' else '0'
    return ''.join(notState)  

def H(wire, inputState):
    outputState = [hadamardGate(wire, state) for state in inputState]
    return AddDuplicate(list(itertools.chain(*outputState)))
def hadamardGate(wire, state):
    notState = bitFlip(wire,state[1])
    if(state[1][wire]=='0'):
        return (1/np.sqrt(2)*state[0], state[1]),(1/np.sqrt(2)*state[0],notState)
    else:
        return (1/np.sqrt(2)*state[0], notState),(-1/np.sqrt(2)*state[0],state[1])
    
PrettyPrintBinary(H(0,[(1,'0')]))

(0.7071067811865475 |0> + 0.7071067811865475 |1>)


In [7]:
def Phase(wire, theta, inputState):
    # Apply phase gate to each state in inputState
    return [phaseGate(wire, theta, state) for state in inputState]
def phaseGate(wire, theta, state):
    return (state[0], state[1]) if state[1][wire] == '0' else (state[0]*np.exp(1j*theta), state[1])

PrettyPrintBinary(Phase(0,np.pi/2,[(1/np.sqrt(2),'0'),(1/np.sqrt(2),'1')]))

(0.7071067811865475 |0> + (4.329780281177466e-17+0.7071067811865475j) |1>)


In [8]:
def CNOT(controlWire,notWire,inputState):
    return [cnotGate(controlWire,notWire,state) for state in inputState]
def cnotGate(controlWire,notWire,state):
    return state if state[1][controlWire]=='0' else (state[0],bitFlip(notWire,state[1]))

test = [
    (np.sqrt(.25),'00'),
    (np.sqrt(.25),'01'),
    (np.sqrt(.1),'10'),
    (np.sqrt(.4),'11')
]
PrettyPrintBinary(test)
PrettyPrintBinary(CNOT(0,1,test))

(0.5 |00> + 0.5 |01> + 0.31622776601683794 |10> + 0.6324555320336759 |11>)
(0.5 |00> + 0.5 |01> + 0.31622776601683794 |11> + 0.6324555320336759 |10>)


In [9]:
def ReadInputString(myInput_lines):
    myInput=[]
    myInput_lines=myInput_lines.split('\n')
    myInput_lines = [ i for i in myInput_lines if i!='']
    numberOfWires=int(myInput_lines[0])
    for line in myInput_lines[1:]:
        myInput.append(line.split())
    return (numberOfWires,myInput)

def RunCircuit(numWires,circuitList):
    state = [(1,'0'*numWires)]
    for element in circuitList:
        if element[0]=='H':
            state = H(int(element[1]),state)
        elif element[0]=='P':
            state = Phase(int(element[1]),float(element[2]),state)
        else:
            state = CNOT(int(element[1]),int(element[2]),state)
    return state


In [10]:
numberOfWires,myInput=ReadInputString(open('example.circuit').read())
test = RunCircuit(numberOfWires,myInput)
PrettyPrintBinary(test)
print("Passed example.circuit?: ", test==[((0.4999999999999999+0j), '0000'), ((0.4999999999999999+0j), '0011'), ((0.4502235511763384+0.21748276705561506j), '1100'), ((0.4502235511763384+0.21748276705561506j), '1111')])
numberOfWires,myInput=ReadInputString('''
3
H 1
H 2
P 2 0.3
CNOT 2 1
H 1
H 2
CNOT 2 0
''')

test = RunCircuit(numberOfWires,myInput)
print("Text example:")
PrettyPrintInteger(test)

(0.4999999999999999 |0000> + 0.4999999999999999 |0011> + (0.4502235511763384+0.21748276705561506j) |1100> + (0.4502235511763384+0.21748276705561506j) |1111>)
Passed example.circuit?:  True
Text example:
((0.9776682445628027+0.14776010333066972j) |0> + (0.022331755437197065-0.14776010333066972j) |5>)


In [11]:
#TODO: RUN rand.circuit

## Simulator M
### SImulator M-a

In [34]:
def tensorMe(matrices):
    result = matrices[0]
    for matrix in matrices[1:]:
        result = np.kron(result, matrix)
    return result

def HadamardArray(wire, numWires):
    H_matrix = 1/np.sqrt(2)*np.array([[1, 1], [1, -1]])
    return tensorMe([np.identity(2)] * (wire-1) + [H_matrix] + [np.identity(2)]*(numWires-wire))

def PhaseArray(wire,numWires,theta):
    P_matrix = np.array([[1,0],[0,np.exp(theta*1j)]])
    return tensorMe([np.identity(2)] * (wire-1) + [P_matrix] + [np.identity(2)]*(numWires-wire))

def CNOTArray(controlWire, notWire, numWires):
    C_matrix_up = np.array([[1, 0, 0, 0],  # bit 0 is control bit, bit 1 is not bit
                            [0, 1, 0, 0],
                            [0, 0, 0, 1],
                            [0, 0, 1, 0]])
    C_matrix_down = np.array([[1, 0, 0, 0],
                              [0, 0, 0, 1],
                              [0, 0, 1, 0],
                              [0, 1, 0, 0]])
    if controlWire < notWire:
        return tensorMe([np.identity(2)] * controlWire + [C_matrix_up] + [np.identity(2)] * (numWires - controlWire - 2))
    else:
        return tensorMe([np.identity(2)] * notWire + [C_matrix_down] + [np.identity(2)] * (numWires - notWire - 2))

state = np.array([[x] for x in range(8)])

print(CNOTArray(1,0,3) @ state)
print(CNOTArray(0,1,3) @state)

[[0.]
 [1.]
 [6.]
 [7.]
 [4.]
 [5.]
 [2.]
 [3.]]
[[0.]
 [1.]
 [2.]
 [3.]
 [6.]
 [7.]
 [4.]
 [5.]]
